In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
import torchvision
from torchvision import models

In [2]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
])

In [3]:
train_path = '/home/gpaudel/Documents/DL/eg/splited/train'
test_path = '/home/gpaudel/Documents/DL/eg/splited/val'

In [4]:
train_dataset = ImageFolder(train_path, transform=transform)
test_dataset = ImageFolder(test_path, transform=transform)

In [5]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [10]:
class ConvNet(nn.Module):
     def __init__(self):
         super(ConvNet, self).__init__()
        
         self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
         self.relu1 = nn.ReLU()
         self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
         self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
         self.relu2 = nn.ReLU()
         self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
         self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
         self.relu3 = nn.ReLU()
         self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        
         self.fc1 = nn.Linear(64 * 16 * 16, 128)
         self.relu4 = nn.ReLU()
        
         self.fc2 = nn.Linear(128, 5)  
        
     def forward(self, op):
         op = self.pool1(self.relu1(self.conv1(op)))
         op = self.pool2(self.relu2(self.conv2(op)))
         op = self.pool3(self.relu3(self.conv3(op)))
         op = op.view(op.size(0), -1)
         op = self.relu4(self.fc1(op))
         op = self.fc2(op)
         return op

model = ConvNet()
def save_checkpoint(state,filename = "myfirstcheckpoint.pth"):
     torch.save(state, filename)
     
def load_checkpoint(checkpoint):
     model.load_state_dict(checkpoint['state_dict'])

In [7]:
print(model)

ConvNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu3): ReLU()
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=16384, out_features=128, bias=True)
  (relu4): ReLU()
  (fc2): Linear(in_features=128, out_features=5, bias=True)
)


In [11]:
# model = ConvNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

best_accuracy = 0.0
best_epoch = 0

num_epochs = 10
# load_model = True
# load_model(torch.load("myfirstcheckpoint.pth"))

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_correct = 0
    if epoch == 5:
        checkpoint = {'state_dict': model.state_dict()}
        save_checkpoint(checkpoint)
    
    for images, labels in train_loader:
        outputs = model(images)
        loss = loss_function(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        train_correct += (predicted == labels).sum().item()

    train_loss /= len(train_loader)
    train_accuracy = 100.0 * train_correct / len(train_dataset)
    # train_accuracy = train_correct / train_dataset

    
    model.eval()
    test_correct = 0

    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            test_correct += (predicted == labels).sum().item()

    test_accuracy = 100.0 * test_correct / len(test_dataset)
    # test_accuracy = test_correct / test_dataset

   
    print(f"epoch [{epoch + 1}/{num_epochs}], train loss is: {train_loss:.4f}, train accuracy is: {train_accuracy:.2f}%, test accuracy: {test_accuracy:.2f}%")


epoch [1/10], train loss is: 1.1439, train accuracy is: 54.07%, test accuracy: 67.92%
epoch [2/10], train loss is: 0.7431, train accuracy is: 72.79%, test accuracy: 68.32%
epoch [3/10], train loss is: 0.5416, train accuracy is: 80.77%, test accuracy: 77.82%
epoch [4/10], train loss is: 0.3835, train accuracy is: 86.95%, test accuracy: 76.24%
epoch [5/10], train loss is: 0.2364, train accuracy is: 91.79%, test accuracy: 75.64%
epoch [6/10], train loss is: 0.1404, train accuracy is: 95.42%, test accuracy: 76.24%
epoch [7/10], train loss is: 0.0753, train accuracy is: 97.39%, test accuracy: 75.05%
epoch [8/10], train loss is: 0.0551, train accuracy is: 98.30%, test accuracy: 74.46%
epoch [9/10], train loss is: 0.0401, train accuracy is: 98.61%, test accuracy: 76.04%
epoch [10/10], train loss is: 0.0403, train accuracy is: 98.52%, test accuracy: 74.65%
